# IMDB 영화 리뷰 감성 분석

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3) #tsv 가지고 올떄

In [26]:
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [28]:
df.review[0][:1000] # 타입이 object여서 바꿔야한다

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [32]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

## 1. 전처리

In [5]:
#<br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

In [6]:
# 구둣점, 숫자 제거 --> 영어 이외의 문자 공백
import re
df.review = df.review.apply(lambda x : re.sub('[^A-Za-z]',' ',x))

In [31]:
df.review[0][:1000] # 타입이 object여서 바꿔야한다

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

## 2. 학습 및 테스트 데이터 셋으로 분리

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    df.review, df.sentiment, test_size= 0.2, #20000개는 train(훈련) 5000개는 test
    stratify=df.sentiment, random_state=2021
)# straify --> 균등하게 분류
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((20000,), (5000,), (20000,), (5000,))

## 3. 텍스트 변환

### countVectrizer, ngram_range=(1,1)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
cvect1 = CountVectorizer(stop_words='english')
X_train_cv1 = cvect1.fit_transform(X_train)
X_test_cv1 = cvect1.fit_transform(X_test)#이렇게하면 모델 엉망이되니 하면안된다

In [37]:
X_train_cv1.shape, X_test_cv1.shape #이렇게하면 모델 엉망이되니 하면안된다

((20000, 66971), (5000, 37187))

In [38]:
# train, test data set에 있는 동일한 단어에 대해서 동일한 값으로 변환
cvect1 = CountVectorizer(stop_words='english') 
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)

In [39]:
X_train_cv1.shape, X_test_cv1.shape # 서로 같은 문자키값을 가진다

((20000, 66971), (5000, 66971))

### countVectrizer, ngram_range=(1,2) 일때

In [48]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape 


((20000, 1461350), (5000, 1461350))

### tfd , ngra_range(1,1)

In [58]:
tvect1 = TfidfVectorizer(stop_words='english')
tvect1.fit(X_train)
X_train_tv1 = tvect1.transform(X_train)
X_test_tv1 = tvect1.transform(X_test)
X_train_tv1.shape, X_test_tv1.shape 


((20000, 66971), (5000, 66971))

### tfd , ngra_range(1,2)

In [63]:
tvect2 = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)
X_train_tv2.shape, X_test_tv2.shape 


((20000, 1461350), (5000, 1461350))

## 4.로지스틱 회귀 모델로 학습(훈련)

In [40]:
from sklearn.linear_model import LogisticRegression

In [42]:
# 객체 생성 및 훈련 - 훈련 데이터의 X와 Y
lr1 = LogisticRegression(max_iter=500)
%time lr1.fit(X_train_cv1, Y_train) #데이터와 결과값

Wall time: 4.83 s


LogisticRegression(max_iter=500)

In [51]:
lr2 = LogisticRegression(max_iter=500)
%time lr2.fit(X_train_cv2, Y_train) #데이터와 결과값

Wall time: 54.1 s


LogisticRegression(max_iter=500)

In [59]:
lr3 = LogisticRegression(max_iter=500)
%time lr3.fit(X_train_tv1, Y_train) #데이터와 결과값

Wall time: 1.61 s


LogisticRegression(max_iter=500)

In [64]:
lr4 = LogisticRegression(max_iter=500)
%time lr4.fit(X_train_tv2, Y_train) #데이터와 결과값

Wall time: 20.6 s


LogisticRegression(max_iter=500)

In [43]:
# 테스트 데이터에 대해 예측 - 테스트 데이터의X 만
pred1 =lr1.predict(X_test_cv1) #에측

In [52]:
pred2 =lr2.predict(X_test_cv2) #에측

In [61]:
pred3 = lr3.predict(X_test_tv1)

In [65]:
pred4 = lr4.predict(X_test_tv2)

## 5. 평가

In [44]:
from sklearn.metrics import accuracy_score

In [45]:
score1 = accuracy_score(Y_test, pred1)
score1

0.8832

In [53]:
score2 = accuracy_score(Y_test, pred2)
score2

0.8914

In [62]:
score3 = accuracy_score(Y_test, pred3)
score3

0.893

In [66]:
score4 = accuracy_score(Y_test, pred4)
score4

0.883

## 6. 비지도 학습 기반 감성분석

### 감성어휘 사전

In [68]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\TH\AppData\Roaming\nltk_data...


True

In [84]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sanalyzer = SentimentIntensityAnalyzer()

In [70]:
senti_score = sanalyzer.polarity_scores(df.review[0]) #df 는 위에 정제된 데이터
senti_score  #리뷰에 대한 긍정 부정을 확인

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}

In [71]:
df.sentiment[0] #긍정인데 리뷰만 보고 부정으로 확인하는 경우도있다

1

In [86]:
# senti_score 의 compound 값이 0.1 보다 크면 긍정(1) 낮으면 부정(0)
def get_sentiment(sanalyzer, review, threshold): #리뷰와 임계치
    score = sanalyzer.polarity_scores(review)
    compound_score = score['compound']
    final_score = 1 if compound_score >= threshold else 0
    return final_score

In [87]:
df['vader_pred'] = df.review.apply(lambda x : get_sentiment(sanalyzer, x, 0.1))

In [89]:
accuracy_score(df.sentiment, df.vader_pred)

0.69556

In [90]:
for threshold in [0, 0.1, 0.2, 0.3]:
    df['vader_pred'] = df.review.apply(lambda x : get_sentiment(sanalyzer, x, threshold))
    score = accuracy_score(df.sentiment, df.vader_pred)
    print(f'thereshold는 {threshold} : {score}')

thereshold는 0 : 0.69104
thereshold는 0.1 : 0.69556
thereshold는 0.2 : 0.69952
thereshold는 0.3 : 0.70264
